# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверка гипотезы:

Семейное положение и количество детей клиента влияет на факт погашения кредита в срок.

**Ход исследования**

Статистика о платёжеспособности клиентов представлена банком в файле `data.csv`.
О качестве данных ничего не известно. Поэтому перед проверкой гипотезs понадобится обзор данных.

Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки по возможности исправим самые критичные ошибки данных.
 
Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотезы.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
# получение общей информации о данных в таблице df
display(df.shape)
df.info()

(21525, 12)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
# Описательная статистика по числовым данным
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В таблице 12 колонок и 21525 строк.

Тип данных: 7 колонок - `int64`, `float64`;  5 колонок - `object`.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Количество значений в колонках различается, значит, в данных есть пропущенные значения.

**В колоноках выявлены следующие нарушения:**

1. Отрицательные значения в количество детей в семье.
2. Минимальное значение возраста = 0.
3. Минимальное(отрицательное) и максимальное(более 1000 лет) значение трудового стажа в днях.
4. Строчные буквы сочетаются с прописными в значениях уровня образования.
5. Цели кредита не приведены к однородным значениям.

В каждой строке таблицы представлены данные о заемщиках. Одна часть колонок описывает семейное положение, наличие детей, другая часть колонок - образование, стаж, тип занятости и третья - кредитную историю в банке.

Предварительно можно утверждать, что данных достаточно для проверки гипотезы. Но встречаются некорректные значения и пропуски в данных.

Требуется, по необходимости, устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков `days_employed` равно количеству пропусков `total_income`. Есть предположение, что это одни и теже строки. Проверим совпадение значений между собой по строкам.

In [7]:
# Какие будут уникальные значения в колонке total_income если выбрать пропуски в days_employed
df.query('days_employed != days_employed').total_income.unique()

array([nan])

**Вывод**

Пропуски совпадают полностью. Поскольку обе колонки это расчетные показатели, значит нужно делать запрос разработчикам.

Пропуски `total_income` заменим средним значением, в зависимости от типа занятости.

In [8]:
# Получим медиану среднего дохода(total_income) в зависимости от типа занятости(income_type)
total_income_median = df.groupby('income_type', as_index=False)\
                        .agg({'total_income':'median', })\
                        .rename(columns={'total_income':'median'})
total_income_median

,income_type,median
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


In [9]:
# объединим основную таблицу df с медианными значениями total_income_median
df = df.merge(total_income_median, on='income_type')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,median
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,142594.396847


In [10]:
# заполним пропуски из новой колонки с медианными значениями
df['total_income'] = df['total_income'].fillna(df['median'])

In [11]:
# удалим колонку с медианными значениями 
df = df.drop(columns = ['median'],axis = 1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование


In [12]:
# поверим пропуски в total_income
df['total_income'].isna().sum()

0

**Разберемся с колонкой `days_employed`**

In [14]:
# сгруппирум данные по отрицательным значениям стажа
df.query('days_employed < 0')['income_type'].value_counts()

сотрудник          10014
компаньон           4577
госслужащий         1312
студент                1
в декрете              1
предприниматель        1
Name: income_type, dtype: int64

In [15]:
# сгруппирум данные по положительным значениям стажа
df.query('days_employed >= 0')['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Вывод: положительный стаж указан у групп `пенсионер` и `безработный`, где стаж не меняется. И отрицательный у остальных групп где клиенты работают и стаж увеличивается.

In [16]:
# сгруппирум данные по стажу > 0 и переведем в годовой стаж
df.query('days_employed > 0').groupby('income_type')['days_employed'].mean()/365

income_type
безработный    1003.873021
пенсионер      1000.009565
Name: days_employed, dtype: float64

Возможно размерность стажа для пенсионеров и безработных указана не в днях а в часах, проверим.

In [17]:
# сгруппирум данные по стажу > 0 и переведем в годовой стаж
df.query('days_employed > 0').groupby('income_type')['days_employed'].mean()/365/24

income_type
безработный    41.828043
пенсионер      41.667065
Name: days_employed, dtype: float64

Стаж 41 год для пенсионера выглядит более реально.
Заменим в таблице эти значения на дни.

In [18]:
# разделим значения с аномальным стажем на 24
def hours_to_days(row):
    if row > 0:
        return row/24
    else:
        return row

In [19]:
# Применим функцию к колонке days_employed
df['days_employed'] = df['days_employed'].apply(hours_to_days)

Поскольку далее в исследовании данные по стажу не понадобятся, заменим минус на плюс и заполним пропуски медианным значением

In [20]:
# данные с отрицательным стажем переведем в положительный и заменим пропуски на медиану
df['days_employed'] = df['days_employed'].abs()
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

In [21]:
# Проверим пропуски
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

Пропуски значений обнаружены в числовых колонках `days_employed` и `total_income` и совпадают полностью. Поскольку обе колонки это расчетные показатели, значит нужно делать запрос разработчикам.

Пропуски `total_income` заменили медианным значением, в зависимости от типа занятости.
Из значений `days_employed` убрали экстремально большие показатели трудового стажа, поменяли знак - на + и запонили пропуски медианным значением.

### Замена типа данных

In [22]:
# Замена типа float на int в колонке `total_income`
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование


**Вывод**

В df представлены 2 колонки `days_employed` и `total_income` в которых можно заменить тип `float` на `int`.

Замена тип `float` на `int` сделает значения более читаемыми и сэкономит ресурсы.

### Обработка дубликатов

In [23]:
# подсчёт явных дубликатов
df.duplicated().sum()

54

In [24]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

In [25]:
# проверка явных дубликатов
df.duplicated().sum()

0

In [26]:
# Поиск неявных дубликатов в колонке 'education'
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Неоконченное высшее', 'Высшее', 'Начальное', 'УЧЕНАЯ СТЕПЕНЬ',
       'НАЧАЛЬНОЕ', 'ученая степень', 'Ученая степень'], dtype=object)

In [27]:
# приведем значения 'education' к нижнему регистру
df['education'] = df['education'].str.lower()

In [28]:
# Проверка неявных дубликатов 'education'
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

В таблице было найдено 54 явных дубликата, строк полностью повторяющих друг друга. Дубликаты были удалены из таблицы и обновлены индексы. Возникновение их нужно адресовать разработчикам.

Так же было выявлено написание одних и тех же значений в разном регистре в колонке `education`. Все значения были приведены к однообразию - строковым буквам. 

**Проверим остальные колонки на соответствие данным**

In [29]:
# значения колонки children
df.groupby('children').agg({'children':'count'})

,children
children,
-1,47
0,14107
1,4809
2,2052
3,330
4,41
5,9
20,76


In [30]:
# значения -1 и 20 выбиваются из общего набора данных, в целом их количество не превышает 1%
# заменим -1 на 1, 20 на 2
# проверим результат
df.loc[df['children']==-1, 'children'] = 1
df.loc[df['children']==20, 'children'] = 2
df.groupby('children').agg({'children':'count'})

,children
children,
0,14107
1,4856
2,2128
3,330
4,41
5,9


In [31]:
# значения колонки gender
df.groupby('gender').agg({'gender':'count'})

,gender
gender,
F,14189
M,7281
XNA,1


In [32]:
# непонятный пол XNA на одной строке, удалим ее с обновлением индексов и проверим результат
df = df.loc[df['gender'] != 'XNA'].reset_index(drop=True)
df['gender'].unique()

array(['F', 'M'], dtype=object)

In [33]:
# значения колонки family_status
df.groupby('family_status').agg({'family_status':'count'})

,family_status
family_status,
Не женат / не замужем,2810
в разводе,1195
вдовец / вдова,959
гражданский брак,4162
женат / замужем,12344


In [34]:
# приведем к нижнему регистру
df['family_status'] = df['family_status'].str.lower()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'в разводе',
       'не женат / не замужем', 'вдовец / вдова'], dtype=object)

In [35]:
# значения колонки income_type
df.groupby('income_type').agg({'income_type':'count'})

,income_type
income_type,
безработный,2
в декрете,1
госслужащий,1457
компаньон,5079
пенсионер,3837
предприниматель,2
сотрудник,11091
студент,1


Замечаний нет

In [36]:
# проверим нулевое значение в колонке возраст dob_years
df.query('dob_years == 0').agg({'dob_years':'count'})

dob_years    101
dtype: int64

Возраст для проверки гипотезы не нужен, менять ничего не будем

In [37]:
# значения колонки purpose
df.groupby('purpose').agg({'purpose':'count'})

,purpose
purpose,
автомобили,478
автомобиль,494
высшее образование,452
дополнительное образование,460
жилье,646
заняться высшим образованием,496
заняться образованием,408
на покупку автомобиля,472
на покупку подержанного автомобиля,478


В колонке `purpose` требуется переименовать цели кредита по похожим значениям, поможет лемматизация.

### Лемматизация

In [38]:
# загрузка библиотеки pymystem3
from pymystem3 import Mystem
m = Mystem()

In [39]:
# лемматизируем колонку 'purpose'
purposes = df['purpose'].unique()
display(len(purposes))
[m.lemmatize(purpose) for purpose in purposes]

38

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль', '\n'],
 ['получение', ' ', 'образование', '\n'],
 ['заниматься', ' ', 'образование', '\n'],
 ['автомобиль', '\n'],
 ['свадьба', '\n'],
 ['получение', ' ', 'дополнительный', ' ', 'образование', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['операция', ' ', 'с', ' ', 'жи

**Вывод**

В колонке `purpose` выявлено 38 различных целей кредита, смысл многих повторяется. Для дальнейшей работы сократим количество этих целей объеденив их в категории по смыслу.

### Категоризация данных

In [40]:
# разбиваем на категории по леммам
def purpose_rename(purpose):
    lemmas = m.lemmatize(purpose)
    for row in lemmas:
        if 'ремонт' in row:
            return 'ремонт жилья'
        if 'жилье' in row or 'недвижимость' in row:
            return 'покупка недвижимости'
        if 'строительство' in row:
            return 'строительство недвижимости'
        if 'образование' in row:
            return 'получение образования'
        if 'автомобиль' in row:
            return 'покупка автомобиля'
        if 'свадьба' in row:
            return 'организация свадьбы'

In [41]:
# добавим колонку'purpose_rename' с категориями
df['purpose_rename']= df['purpose'].apply(purpose_rename)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_rename
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,покупка автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования
4,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования


In [42]:
# проверим результат
df.groupby('purpose_rename').agg({'purpose_rename':'count'})

,purpose_rename
purpose_rename,
организация свадьбы,2335
покупка автомобиля,4308
покупка недвижимости,8327
получение образования,4014
ремонт жилья,607
строительство недвижимости,1879


**Вывод**

В результате категоризации были выделены 6 основных целей кредита:
 - ремонт жилья
 - покупка недвижимости
 - строительство недвижимости
 - получение образования
 - покупка автомобиля
 - организация свадьбы
 
Их в свою очередь можно разделить на 3 категорий:
 - долгосрочные - покупка недвижимости,
 - среднесрочные - получение образования, покупка автомобиля,
 - краткосрочные - ремонт жилья, строительство недвижимости, организация свадьбы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [43]:
# сначала посмотрим общие показатели по задолженности
df_debt_share = df['debt'].sum()/df['debt'].count()*100
df_debt_share.round(2)

8.11

**Средняя задолженность по всей таблице составляет 8,11%**

Сравнивать будем с этим показателем.

In [44]:
# сгруппируем клиентов по наличию детей и задолженности по возврату кредитов 'debt'
# и посчитаем долю задолженности по каждой группе 
children_grouped = df.groupby('children').agg({'debt':['count', 'sum']})
children_grouped['debt_share'] = children_grouped['debt']['sum']/children_grouped['debt']['count']*100
children_grouped.round(2)

debt       debt_share
          count   sum           
children                        
0         14106  1063       7.54
1          4856   445       9.16
2          2128   202       9.49
3           330    27       8.18
4            41     4       9.76
5             9     0       0.00

**Вывод**

Семьи без детей имеют задолженность по возврату кредитов 7,54%.
В семьях где 1 ребенок задолженность уже больше - 9.16%, 2 ребенка - 9,49%. 
Дальше, где 3 и более ребенка процент задолженности снижается, но количество таких значений мало и не значительно влияет на общую картину.

Значит можно сделать вывод, что наличие детей и их количество влияет на выплату кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
# сгруппируем клиентов по семейному положению и посчитаем задолженность по каждой группе
family_status_grouped = df.groupby('family_status').agg({'debt':['count', 'sum']})
family_status_grouped['debt_share'] = family_status_grouped['debt']['sum']/family_status_grouped['debt']['count']*100
family_status_grouped.round(2)

debt      debt_share
                       count  sum           
family_status                               
в разводе               1195   85       7.11
вдовец / вдова           959   63       6.57
гражданский брак        4162  388       9.32
женат / замужем        12344  931       7.54
не женат / не замужем   2810  274       9.75

**Вывод**

Клиенты котрые были или находятся в официальном браке являются более ответственными плательщиками: 
- женат/замужем, 
- в разводе, 
- вдовец / вдова,

По сравнению с клиентами не оформлявшими официальнй брак:
- Не женат / не замужем, 
- гражданский брак.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
# посмотрим описательную статистику по среднемесячному доходу 
df.total_income.describe()

count    2.147000e+04
mean     1.652936e+05
std      9.815539e+04
min      2.066700e+04
25%      1.076522e+05
50%      1.425940e+05
75%      1.957518e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [47]:
# среднее значение дохода составляет около 151 тыс, медианное значение около 136 тысяч
# разобъем значения дохода на несколько групп
def income_rate(total_income):
    if total_income < 100000:
        return 'до 100 тыс'
    if total_income < 200000:
        return 'от 100 до 200 тыс'
    if total_income < 500000:
        return 'от 200 до 500 тыс'
    return 'от 500 тыс'

In [48]:
# добавим колонку с рейтингом по доходу
df['income_rate']= df['total_income'].apply(income_rate)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_rename,income_rate
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка недвижимости,от 200 до 500 тыс
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,покупка автомобиля,от 100 до 200 тыс
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка недвижимости,от 100 до 200 тыс
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,получение образования,от 200 до 500 тыс
4,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,получение образования,от 100 до 200 тыс


In [49]:
# сгруппируем клиентов по рейтингу дохода и посчитаем задолженность по каждой группе
income_grouped = df.groupby('income_rate').agg({'debt':['count','sum']})
income_grouped['debt_share'] = income_grouped['debt']['sum']/income_grouped['debt']['count']*100
income_grouped.round(2)

debt       debt_share
                   count   sum           
income_rate                              
до 100 тыс          4463   354       7.93
от 100 до 200 тыс  11941  1029       8.62
от 200 до 500 тыс   4844   344       7.10
от 500 тыс           222    14       6.31

**Вывод**

Самые недобросовестные плательщики находятся в группе с доходом от 100 до 200 тыс в месяц, те приближенные к среднему и медианному значению. При увеличении дохода доля задолженности падает. Так же лучше платят клиенты с доходом менее 100 тыс.

- Как разные цели кредита влияют на его возврат в срок?

In [50]:
# сгруппируем клиентов по цели кредита и задолженности по возврату кредита
# и посчитаем долю задолженности по каждой группе клиентов
purpose_grouped = df.groupby('purpose_rename').agg({'debt':['count', 'sum']})
purpose_grouped['debt_share'] = purpose_grouped['debt']['sum']/purpose_grouped['debt']['count']*100
purpose_grouped.round(2)

debt      debt_share
                           count  sum           
purpose_rename                                  
организация свадьбы         2335  186       7.97
покупка автомобиля          4308  403       9.35
покупка недвижимости        8327  603       7.24
получение образования       4014  370       9.22
ремонт жилья                 607   35       5.77
строительство недвижимости  1879  144       7.66

**Вывод**

Хуже всего платят клиенты приобретающие автомобиль и получающие образование. Предположительно, что эти кредиты относятся к среднесрочным 3-5 лет. Лучше платят по кредитам которые можно отнести к краткосрочным кредитам: ремонт, строительство, свадьба. И лучше платят долгосрочные кредиты покупка недвижимости(ипотека).

## Шаг 4. Общий вывод

Лучшего заемщика можно описать следующими показателями:

Клиент состоит или состоял в официальном браке, не имеет детей, с доходом до 100 тыс или более 200 тыс, берущий краткосрочный кредит или покупающий недвижимость.

Менее добросовестного клиента можно описать так:

Не состоит в официальном браке, имеет детей, доход 100-200 тыс, планирует приобретать автомобиль или получить образование.